<a href="https://colab.research.google.com/github/jiel30/Completed/blob/master/sentence_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import datasets
import tensorflow as tf
torch.cuda.is_available()

True

In [5]:
!unzip train_dataset.zip
!pip3 install datasets
!pip3 install transformers

Archive:  train_dataset.zip
   creating: train_dataset/
  inflating: __MACOSX/._train_dataset  
  inflating: train_dataset/.DS_Store  
  inflating: __MACOSX/train_dataset/._.DS_Store  
   creating: train_dataset/BQ/
  inflating: __MACOSX/train_dataset/._BQ  
   creating: train_dataset/LCQMC/
  inflating: __MACOSX/train_dataset/._LCQMC  
   creating: train_dataset/OPPO/
  inflating: __MACOSX/train_dataset/._OPPO  
  inflating: train_dataset/BQ/.DS_Store  
  inflating: __MACOSX/train_dataset/BQ/._.DS_Store  
  inflating: train_dataset/BQ/dev.txt  
  inflating: __MACOSX/train_dataset/BQ/._dev.txt  
  inflating: train_dataset/BQ/train.txt  
  inflating: __MACOSX/train_dataset/BQ/._train.txt  
  inflating: train_dataset/BQ/test.txt  
  inflating: __MACOSX/train_dataset/BQ/._test.txt  
  inflating: train_dataset/LCQMC/.DS_Store  
  inflating: __MACOSX/train_dataset/LCQMC/._.DS_Store  
  inflating: train_dataset/LCQMC/dev.txt  
  inflating: __MACOSX/train_dataset/LCQMC/._dev.txt  
  inflating

In [2]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

checkpoint = 'bert-base-chinese'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = 2)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
sequences = [
    "婴儿吃什么蔬菜好",
    "婴儿吃什么绿色蔬菜好",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
batch['labels'] = torch.tensor([1, 1])  # tokenizer出来的结果是一个dictionary，所以可以直接加入新的 key-value

##这个label表示两者都是positive的

optimizer = AdamW(model.parameters())
loss = model(**batch).loss  #这里的 loss 是直接根据 batch 中提供的 labels 来计算的，回忆：前面章节查看 model 的输出的时候，有loss这一项
loss.backward()
optimizer.step()

In [4]:
#处理一下data
# ROOT = '/Users/lijie/Documents/nlp/data/train_dataset'
ROOT = '/content/train_dataset'
names = ['BQ','LCQMC','OPPO']
# 将其中所有的data处理成一个dict：{'sentence1': 'xxx', 'sentence2': 'yyyy','label':0}
import pathlib
from pathlib import Path
import os

#每个数据集中有train.txt, dev.txt, test.txt (OPPO中没有test.txt)
def process_data(path = ROOT,names = names):
    paths = [Path(path)/Path(name) for name in names]
    dataset_dict = {'train':{'sentence1':[],'sentence2':[],'label':[]},
                    'validation':{'sentence1':[],'sentence2':[],'label':[]},
                    'test':{'sentence1':[],'sentence2':[],'label':[]}}
    for p in paths:
        for file in [ p/Path(file) for file in os.listdir(p)]:
            if 'train.txt' in str(file):
                with open(file,'r') as f:
                    # print(file)
                    for line in f.readlines():
                        # print(process_text(line))
                        s1,s2,label = process_text(line)
                        dataset_dict['train']['sentence1'].append(s1)
                        dataset_dict['train']['sentence2'].append(s2)
                        dataset_dict['train']['label'].append(int(label))
            elif 'dev.txt' in str(file):
                with open(file,'r') as f:
                    for line in f.readlines():
                        # print(process_text(line))
                        s1,s2,label = process_text(line)
                        dataset_dict['validation']['sentence1'].append(s1)
                        dataset_dict['validation']['sentence2'].append(s2)
                        dataset_dict['validation']['label'].append(int(label))
            elif 'test.txt' in str(file):
                with open(file,'r') as f:
                    for line in f.readlines():
                        s1,s2,label = process_text(line)
                        dataset_dict['test']['sentence1'].append(s1)
                        dataset_dict['test']['sentence2'].append(s2)
                        dataset_dict['test']['label'].append(int(label))
    return dataset_dict

def process_text(text):
    return text.replace('\n','').split('\t')

In [5]:
dataset = process_data()

In [6]:
train_encodings = tokenizer(dataset['train']['sentence1'],dataset['train']['sentence2'],truncation=True, padding=True)
val_encodings = tokenizer(dataset['validation']['sentence1'],dataset['validation']['sentence2'], truncation=True, padding=True)
# test_encodings = tokenizer(dataset['test']['sentence1'],dataset['test']['sentence2'], truncation=True, padding=True)

In [7]:
train_label = dataset['train']['label']
val_label = dataset['validation']['label']
test_label = dataset['test']['label']

In [9]:
import torch

class DDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = DDataset(train_encodings, train_label)
val_dataset = DDataset(val_encodings, val_label)
# test_dataset = DDataset(test_encodings, test_label)

In [10]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

***** Running training *****
  Num examples = 505939
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 94866


Step,Training Loss
10,1.014400
20,0.830000
30,0.722100
40,0.690100
50,0.721300
60,0.678900
70,0.670900
80,0.701400
90,0.681700
100,0.659700


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
